# Bedrock with LangChain using a Prompt that includes Context

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Setup

In [1]:
import json
import os
import sys
import boto3

# Add any necessary paths for custom modules
module_path = ".."
sys.path.append(os.path.abspath(module_path))

from labutils import bedrock, print_ww

# Uncomment and edit the following lines for your AWS setup
# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g., "us-west-2"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [2]:
%pip install -U langchain-aws
%pip install pydantic==1.10.7
%pip install anthropic

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Reference the Bedrock LLM

In [3]:
from langchain_aws import BedrockLLM

inference_modifier = {
    'max_tokens_to_sample': 4096, 
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"]
}

textgen_llm = BedrockLLM(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock, 
    model_kwargs=inference_modifier 
)


## Create a LangChain custom prompt template

In [4]:
from langchain.prompts import PromptTemplate

# Create a prompt template with multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""
    Human: Create an apology email from the Service Manager {customerServiceManager} to {customerName} in response to the following feedback that was received from the customer: 
    <customer_feedback>
    {feedbackFromCustomer}
    </customer_feedback>

    Assistant:"""
)

# Fill in the template with specific values
prompt = multi_var_prompt.format(
    customerServiceManager="Swami", 
    customerName="John Doe", 
    feedbackFromCustomer="""Hello Swami,
         I am very disappointed with the recent experience I had when I called your customer support.
         I was expecting an immediate call back but it took three days for us to get a call back.
         The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
         We are very unhappy with the response provided and may consider taking our business elsewhere."""
)


### Print the number of tokens in the prompt (for interest/debug)

In [5]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

Our prompt has 125 tokens


In [6]:
print(prompt)


    Human: Create an apology email from the Service Manager Swami to John Doe in response to the following feedback that was received from the customer: 
    <customer_feedback>
    Hello Swami,
         I am very disappointed with the recent experience I had when I called your customer support.
         I was expecting an immediate call back but it took three days for us to get a call back.
         The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
         We are very unhappy with the response provided and may consider taking our business elsewhere.
    </customer_feedback>

    Assistant:


## Invoke the model and print the response

In [7]:
response = textgen_llm.invoke(prompt)

email = response[response.index('\n')+1:]
print_ww(email)



I want to sincerely apologize for the poor service you recently received from our customer support
team. It is clear from your feedback that we failed to meet your expectations and provide the level
of service that you deserve.

I was disappointed to hear that it took three days to receive a callback after your initial request.
That is unacceptable. Our policy is to respond to all customer inquiries within 24 hours. I will be
following up with our support team to understand why there was such a long delay in getting back to
you.

I also want to apologize that the first suggestion provided did not resolve your issue. We strive to
provide accurate troubleshooting advice to fix problems promptly. The fact that it took three days
to ultimately resolve your issue is not at all reflective of our normal service standards.

I want to assure you that this experience is not typical of the service we aim to deliver to our
valued customers like yourself. Your business is important to us and we ne